# **Projet 5 : Segmentez des clients d'un site e-commerce** 

######

----------------------------------------------------------------------------------------------------------------------------------------
## **Partie 1: Réponses aux questions**
----------------------------------------------------------------------------------------------------------------------------------------

#

### **01 - Library**

In [1]:
import sqlite3
import pandas as pd

from datetime import datetime, timedelta

C:\Users\521169\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


###  02 - Data loading

In [12]:
# Connect to the SQLite database
db_path = r"C:\Users\521169\Documents\Openclassrooms\PROJET 5\Base de donnée\olist.db"
db = r"C:\Users\521169\Documents\Openclassrooms\PROJET 5\Base de donnée\P5donneeCSV\base.csv"
conn = sqlite3.connect(db_path)
# Get a cursor object
cursor = conn.cursor()

df = pd.read_csv(db)
df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,seller_id,price,freight_value
0,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72
1,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,22.76
2,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,159.90,19.22
3,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,45.00,27.20
4,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,19.90,8.72


In [3]:
df_orders = pd.read_sql_query("SELECT * from orders", conn)
df_orders = df_orders.drop(columns = ["index"])
MAXDATE = df_orders['order_purchase_timestamp'].max()
MAXDATE

'2018-10-17 17:30:18'

#### derniere commande éffectuée  en **octobre 2018**

In [4]:
df_orders['order_purchase_timestamp'].min()

'2016-09-04 21:15:19'

#### 1re commande éffectuée  en **Septembre 2016**

##

##

### **QUESTION 1 :  En excluant les commandes annulées, quelles sont les commandes récentes de moins de 3 mois que les clients ont reçues avec au moins 3 jours de retard ?**

In [5]:
# En excluant les commandes annulées, quelles sont les commandes récentes de moins de 3 mois que les clients ont reçues avec au moins 3 jours de retard ? 

Query1 = """SELECT 
    o.order_id, 
    o.customer_id, 
    o.order_purchase_timestamp, 
    o.order_estimated_delivery_date, 
    o.order_delivered_customer_date
FROM orders o
WHERE o.order_status != 'canceled' 
AND o.order_delivered_customer_date > o.order_estimated_delivery_date
AND julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date) >= 3 
AND o.order_purchase_timestamp >= date((SELECT MAX(order_purchase_timestamp) FROM orders), '-3 months')"""

Question1 = pd.read_sql_query(Query1, conn)
Question1

,order_id,customer_id,order_purchase_timestamp,order_estimated_delivery_date,order_delivered_customer_date
0,cfa4fa27b417971e86d8127cb688712f,7093250e1741ebbed41f0cc552025fd6,2018-08-16 09:44:23,2018-08-22 00:00:00,2018-08-29 01:41:41
1,234c056c50619f48da64f731c48242b4,44e460a655f7154ccd9faa4dbbbaf68a,2018-08-14 14:49:15,2018-08-23 00:00:00,2018-09-01 18:14:42
2,8ad3f1d0f96992e43566c4c82c9f6c58,948b29e24216a05fea13a18d8db45ea5,2018-07-17 21:25:29,2018-08-03 00:00:00,2018-08-14 04:04:40
3,7f579e203c931f3e8410103359c6d523,d665be250d1c687c58fdea61a9b55a58,2018-08-02 18:09:27,2018-08-09 00:00:00,2018-08-13 20:11:47
4,cb6e441ff2ef574ce08d3709426f88ec,4fb843d304c57182d4aa27bb39ca592b,2018-08-08 19:27:03,2018-08-15 00:00:00,2018-08-18 01:11:58
...,...,...,...,...,...
317,874a7690bc049bd4ce210d195bdfff7b,f6b2c83ea477af36f44a1e2df7320036,2018-08-02 12:06:47,2018-08-08 00:00:00,2018-08-13 15:50:48
318,587e32dd528769d669d539531d32aeb5,0af8a25fb0b6f833724bb5818275807b,2018-08-10 11:46:09,2018-08-28 00:00:00,2018-09-03 09:32:31
319,3adb141ba4bd69dd7fe8d3fb733c6b74,c0539d5c87fc7c97a8418adffe4b45f0,2018-08-14 23:29:21,2018-08-24 00:00:00,2018-08-28 18:02:52
320,4cf09d9e5ebbe0f91ddd7bf9aae891cd,07b6b4fe5fefb948fc76b6d2bdba77d8,2018-07-19 08:37:26,2018-08-10 00:00:00,2018-08-21 01:12:45


#

### **QUESTION 2 : Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100 000 Real sur des commandes livrées via Olist ?**

In [6]:
# Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100 000 Real sur des commandes livrées via Olist ?

query_2 = """

WITH orders_joined as (
SELECT distinct i.seller_id,
i.order_id,
i.price,
o.order_purchase_timestamp 

from order_items as i
INNER JOIN orders as o
on o.order_id = i.order_id
where o.order_status = 'delivered'
order by seller_id, order_purchase_timestamp
),
aggregation as (
SELECT seller_id, sum(price) as total_amount_sold,
count(order_id) as total_items_sold 

FROM orders_joined
group by seller_id
)
select * from aggregation
where total_amount_sold > 100000
order by total_amount_sold desc """

QUESTION2 = pd.read_sql_query(query_2, conn)
QUESTION2

,seller_id,total_amount_sold,total_items_sold
0,4869f7a5dfa277a7dca6462dcf3b52b2,224921.54,1135
1,53243585a1d6dc2643021fd1853d8905,203364.00,348
2,fa1c13f2614d7b5c4749cbc52fecda94,190917.14,579
3,4a3ca9315b744ce9f8e9374361493884,183828.07,1810
4,7e93a43ef30c4f03f38b393420bc753a,165822.49,321
5,da8622b14eb17ae2831f4ac5b9dab84a,141875.38,1371
6,7c67e1448b00f6e969d365cea6b010ab,140998.62,995
7,7a67c85e85bb2ce8582c35f2203ad736,139098.77,1147
8,46dc3b2cc0980fb8ec44634e21d2718e,120671.54,504
9,6560211a19b47992c3666cc44a7e94c0,116660.83,1905


#

### **QUESTION 3 : Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30 produits) ?**

In [9]:
# Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30 produits) ?

query_3 = """

   WITH LatestOrderDate AS (
    SELECT MAX(order_purchase_timestamp) AS MaxDate
    FROM orders
),
SellerActivity AS (
    SELECT
        oi.seller_id,
        MIN(o.order_purchase_timestamp) AS first_sale_date,
        COUNT(oi.order_item_id) AS total_products_sold
    FROM 
        order_items oi
    JOIN 
        orders o ON oi.order_id = o.order_id
    WHERE 
        o.order_status = 'delivered'
    GROUP BY 
        oi.seller_id
),
SellerEngagement AS (
    SELECT
        sa.seller_id,
        sa.total_products_sold,
        julianday((SELECT MaxDate FROM LatestOrderDate)) - julianday(sa.first_sale_date) AS days_since_first_sale
    FROM 
        SellerActivity sa
)
SELECT 
    seller_id,
    total_products_sold,
    days_since_first_sale
FROM 
    SellerEngagement
WHERE 
    days_since_first_sale <= 90 AND
    total_products_sold > 30
ORDER BY
    total_products_sold DESC; """


QUESTION3 = pd.read_sql_query(query_3, conn)
QUESTION3

,seller_id,total_products_sold,days_since_first_sale
0,d13e50eaa47b4cbe9eb81465865d8cfc,68,74.347697
1,81f89e42267213cb94da7ddc301651da,52,70.197986


#

## **Question 4: Question : Quels sont les 5 codes postaux, enregistrant plus de 30 reviews, avec le pire review score moyen sur les 12 derniers mois ?**

In [8]:
# Question 4: Quels sont les 5 codes postaux, enregistrant plus de 30 reviews, avec le pire review score moyen sur les 12 derniers mois ?

query_4 = """
WITH LatestDate AS (
    SELECT MAX(order_purchase_timestamp) AS MaxDate
    FROM orders
)
SELECT 
    c.customer_zip_code_prefix, 
    AVG(r.review_score) AS average_score, 
    COUNT(r.review_id) AS total_reviews
FROM 
    order_reviews r
JOIN 
    orders o ON r.order_id = o.order_id
JOIN 
    customers c ON o.customer_id = c.customer_id,
    LatestDate
WHERE 
    datetime(o.order_purchase_timestamp) >= datetime(LatestDate.MaxDate, '-12 months')
GROUP BY 
    c.customer_zip_code_prefix
HAVING 
    total_reviews > 30
ORDER BY 
    average_score ASC
LIMIT 5;
"""

df4 = pd.read_sql_query(query_4, conn)
df4


,customer_zip_code_prefix,average_score,total_reviews
0,22753,2.808511,47
1,22770,3.135135,37
2,22793,3.233333,90
3,21321,3.277778,36
4,22780,3.351351,37
